In [1]:
from tg.grammar_ru.ml.components.yandex_storage.s3_yandex_helpers import S3YandexHandler
from tg.grammar_ru.common import Loc
from dotenv import load_dotenv

project_name = 'antecedents_project'
dataset_name = 'antecedents_simple_dataset'
bucket = 'antecedents-simple'
load_dotenv(Loc.root_path / 'environment.env')

True

Загрузка бандла:

In [2]:
#S3YandexHandler.create_bucket(bucket)

In [3]:
bundle_folder = Loc.bundles_path/'antcd/simple/last'
s3path = f'datasphere/{project_name}/datasets/{dataset_name}'

In [7]:
#S3YandexHandler.upload_folder(bucket, s3path, bundle_folder)

In [6]:
from tg.grammar_ru.common import DataBundle
db = DataBundle.load(bundle_folder)
db

{'index': {'shape': (101065, 7), 'index_name': 'sample_id'}, 'pymorphy': {'shape': (570787, 16), 'index_name': 'word_id'}, 'slovnet': {'shape': (570787, 17), 'index_name': 'word_id'}, 'src': {'shape': (570787, 14), 'index_name': None}}